In [54]:
def get_pnl_series(df):
    gross_pnl = []
    net_pnl = []
    current_position = 0
    current_contracts = []

    rows = (
        df["Buy"].astype(int).tolist(),
        df["Sell"].astype(int).tolist(),
        df["Trade Price"].tolist(),
        df["Fees"].tolist()
    )

    for buy_qty, sell_qty, px, fee in zip(*rows):
        gross_pnl_single = fee
        net_pnl_single = 0

        if buy_qty:

            if current_position > 0:
                # Increase pos
                current_contracts.extend([px] * buy_qty)

            elif current_position < 0:
                # Decrease pos / Reverse pos
                is_reverse = buy_qty > abs(current_position)

                closed_qty = abs(current_position if is_reverse else buy_qty)
                closed_contracts = [current_contracts.pop(0) for _ in range(closed_qty)]
                closed_avg_px = sum(closed_contracts) / len(closed_contracts)
                net_pnl_single += (closed_avg_px - px) * 2

                if is_reverse:
                    current_contracts = [px] * (buy_qty + current_position)

            else:
                # Open pos
                current_contracts = [px] * buy_qty

            current_position += buy_qty

        elif sell_qty:

            if current_position < 0:
                # Increase pos
                current_contracts.extend([px] * sell_qty)

            elif current_position > 0:
                # Decrease pos / Reverse pos
                is_reverse = sell_qty > abs(current_position)

                closed_qty = abs(current_position if is_reverse else sell_qty)
                closed_contracts = [current_contracts.pop(0) for _ in range(closed_qty)]
                closed_avg_px = sum(closed_contracts) / len(closed_contracts)
                net_pnl_single += (px - closed_avg_px) * 2

                if is_reverse:
                    current_contracts = [px] * (sell_qty + current_position)

            else:
                # Open pos
                current_contracts = [px] * sell_qty

            current_position -= sell_qty

        gross_pnl.append(net_pnl_single + gross_pnl_single)
        net_pnl.append(net_pnl_single)

    return gross_pnl, net_pnl

In [55]:
import numpy as np
import pandas as pd

df = pd.read_csv(r"D:\UserData\Downloads\tradeblotter_210YY497_2022-05-01_to_2022-07-25.csv", skiprows=2)
df = df[df["Futures Code"] == "MQ"]
df["Positions"] = df["Buy"].cumsum() - df["Sell"].cumsum()
df["Fees"] = df["Posted Commission"] + df["Posted Clearing Fee"] + df["Posted NFA Fee"]
gross_pnl, net_pnl = get_pnl_series(df)
df["Gross PnL"] = gross_pnl
df["Net PnL"] = net_pnl
df["Traded"] = np.where(df["Net PnL"] != 0, df["Buy"] + df["Sell"], 0)
df

,Date,Master Account,Account,Account Type,Contract Description,Exchange,Futures Code,Security Type,Buy,Sell,...,Posted Floor Fee,Floor Fee Currency,Posted NFA Fee,NFA Fee Currency,Unnamed: 21,Positions,Fees,Gross PnL,Net PnL,Traded
0,05/16/2022,,210YY497,F1,JUN 22 CME MCRO NSDQ,CME,MQ,Futures,1.0,0.0,...,0.0,USD,-0.02,USD,NaN,1.0,-0.82,-0.82,0.0,0.0
1,05/16/2022,,210YY497,F1,JUN 22 CME MCRO NSDQ,CME,MQ,Futures,1.0,0.0,...,0.0,USD,-0.02,USD,NaN,2.0,-0.82,-0.82,0.0,0.0
2,05/16/2022,,210YY497,F1,JUN 22 CME MCRO NSDQ,CME,MQ,Futures,0.0,1.0,...,0.0,USD,-0.02,USD,NaN,1.0,-0.82,-52.82,-52.0,1.0
3,05/16/2022,,210YY497,F1,JUN 22 CME MCRO NSDQ,CME,MQ,Futures,0.0,1.0,...,0.0,USD,-0.02,USD,NaN,0.0,-0.82,-114.82,-114.0,1.0
4,05/17/2022,210YJHSI,210YY497,F1,JUN 22 CME MCRO NSDQ,CME,MQ,Futures,0.0,1.0,...,0.0,USD,-0.02,USD,NaN,-1.0,-0.82,-0.82,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2247,07/22/2022,210YJHSI,210YY497,F1,SEP 22 CME MCRO NSDQ,CME,MQ,Futures,1.0,0.0,...,0.0,USD,-0.02,USD,NaN,-13.0,-0.57,527.43,528.0,1.0
2248,07/22/2022,210YJHSI,210YY497,F1,SEP 22 CME MCRO NSDQ,CME,MQ,Futures,1.0,0.0,...,0.0,USD,-0.02,USD,NaN,-12.0,-0.57,524.93,525.5,1.0
2249,07/22/2022,210YJHSI,210YY497,F1,SEP 22 CME MCRO NSDQ,CME,MQ,Futures,1.0,0.0,...,0.0,USD,-0.02,USD,NaN,-11.0,-0.57,521.93,522.5,1.0
2250,07/22/2022,210YJHSI,210YY497,F1,SEP 22 CME MCRO NSDQ,CME,MQ,Futures,11.0,0.0,...,0.0,USD,-0.22,USD,NaN,0.0,-6.27,535.73,542.0,11.0


In [56]:
df[["Date", "Buy", "Sell", "Trade Price", "Positions", "Gross PnL", "Net PnL", "Traded"]]

,Date,Buy,Sell,Trade Price,Positions,Gross PnL,Net PnL,Traded
0,05/16/2022,1.0,0.0,12336.25,1.0,-0.82,0.0,0.0
1,05/16/2022,1.0,0.0,12315.00,2.0,-0.82,0.0,0.0
2,05/16/2022,0.0,1.0,12310.25,1.0,-52.82,-52.0,1.0
3,05/16/2022,0.0,1.0,12258.00,0.0,-114.82,-114.0,1.0
4,05/17/2022,0.0,1.0,12558.50,-1.0,-0.82,0.0,0.0
...,...,...,...,...,...,...,...,...
2247,07/22/2022,1.0,0.0,12355.50,-13.0,527.43,528.0,1.0
2248,07/22/2022,1.0,0.0,12357.00,-12.0,524.93,525.5,1.0
2249,07/22/2022,1.0,0.0,12358.50,-11.0,521.93,522.5,1.0
2250,07/22/2022,11.0,0.0,12424.00,0.0,535.73,542.0,11.0
